Installing and importing required packages

In [1]:
!pip install -q langchain
!pip install -q sentence_transformers
!pip install -q faiss-cpu
!pip install -q streamlit
!pip install joblib

!npm install localtunnel

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tables 3.8.0 requires cython>=0.29.21, which is not installed.
distributed 2023.6.0 requires dask==2023.6.0, but you have dask 2023.11.0 which is incompatible.
python-lsp-black 1.2.1 requires black>=22.3.0, but you have black 0.0 which is incompatible.


In [ ]:
import langchain
import sentence_transformers
import joblib
print(langchain.__version__)
print(sentence_transformers.__version__)
print(joblib.__version__)

0.0.353
2.2.2
1.3.2


In [ ]:
import json
import pandas as pd
import numpy as np

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

Data Collection

In [ ]:
json_file = open('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/medical_meadow_wikidoc_patient_info.json', 'r')
json_data = json.load(json_file)

questions, answers = [], []

for i in json_data:
  questions.append(i['input'])
  answers.append(i['output'])
  print(i, end='\n\n')

Streaming output truncated to the last 5000 lines.
{'instruction': 'Answer this question truthfully', 'input': 'What causes Measles?', 'output': 'Measles is an infectious respiratory disease caused by a virus. It spreads easily from person to person. The main symptom of measles is an itchy skin rash. The rash often starts on the head and moves down the body. The virus normally grows in the cells that line the back of the throat and in the cells that line the lungs.'}

{'instruction': 'Answer this question truthfully', 'input': 'Who is at highest risk for Measles ?', 'output': 'Measles is spread by contact with an infected person, through coughing and sneezing (highly contagious). The disease is highly contagious, and can be transmitted from 4 days prior to the onset of the rash to 4 days after the onset. If one person has it, 90% of their susceptible close contacts will also become infected with the measles virus. The virus resides in the mucus in the nose and throat of the infected pe

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(questions[:10])
print(answers[:10])
print(len(questions))
print(len(answers))

['What are the symptoms of Allergy?', 'What causes Allergy?', 'When to seek urgent medical care when I have Allergy ?', 'What to expect if I have Allergy  (Outlook/Prognosis)?', 'What are the symptoms of Anaphylaxis?', 'What causes Anaphylaxis?', 'When to seek urgent medical care when I have Anaphylaxis ?', 'What to expect if I have Anaphylaxis  (Outlook/Prognosis)?', 'What are the symptoms of Angioedema?', 'What causes Angioedema?']
['Allergy symptoms vary, but may include:\nBreathing problems (coughing, shortness of breath) Burning, tearing, or itchy eyes Conjunctivitis (red, swollen eyes) Coughing Diarrhea Headache Hives Itching of the nose, mouth, throat, skin, or any other area Runny nose Skin rashes Stomach cramps Vomiting Wheezing\nWhat part of the body is contacted by the allergen plays a role in the symptoms you develop. For example:\nAllergens that are breathed in often cause a stuffy nose, itchy nose and throat, mucus production, cough, or wheezing. Allergens that touch the 

In [ ]:
from bs4 import BeautifulSoup

with open('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/TREC-2017-LiveQA-Medical-Train-1.xml', 'r') as f:
  data = f.read()

xml_data = BeautifulSoup(data, 'xml')
nlm_questions = xml_data.find_all('NLM-QUESTION')
for nlm_question in nlm_questions:
  nlm_que = nlm_question.find('MESSAGE').get_text()
  questions.append(nlm_que)
  nlm_ans = nlm_question.find('SUB-QUESTIONS').find('SUB-QUESTION').find('ANSWERS').find('ANSWER').get_text()
  answers.append(nlm_ans)
#print(question1)
#print(answer1)
print(len(answers))
print(len(questions))


6142
6142


In [ ]:
from bs4 import BeautifulSoup

with open('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/TREC-2017-LiveQA-Medical-Train-2.xml', 'r') as f:
  data = f.read()

xml_data = BeautifulSoup(data, 'xml')
nlm_questions = xml_data.find_all('NLM-QUESTION')
for nlm_question in nlm_questions:
  nlm_que = nlm_question.find('MESSAGE').get_text()
  questions.append(nlm_que)
  nlm_ans = nlm_question.find('SUB-QUESTIONS').find('SUB-QUESTION').find('ANSWERS').find('ANSWER').get_text()
  answers.append(nlm_ans)
#print(question1)
#print(answer1)
print(len(answers))
print(len(questions))
questions

6388
6388


['What are the symptoms of Allergy?',
 'What causes Allergy?',
 'When to seek urgent medical care when I have Allergy ?',
 'What to expect if I have Allergy  (Outlook/Prognosis)?',
 'What are the symptoms of Anaphylaxis?',
 'What causes Anaphylaxis?',
 'When to seek urgent medical care when I have Anaphylaxis ?',
 'What to expect if I have Anaphylaxis  (Outlook/Prognosis)?',
 'What are the symptoms of Angioedema?',
 'What causes Angioedema?',
 'When to seek urgent medical care when I have Angioedema ?',
 'What to expect if I have Angioedema  (Outlook/Prognosis)?',
 'What are the symptoms of Asthma?',
 'What causes Asthma?',
 'Who is at highest risk for Asthma ?',
 'When to seek urgent medical care when I have Asthma ?',
 'What to expect if I have Asthma  (Outlook/Prognosis)?',
 'What are the symptoms of dermatitis herpetiformis?',
 'What causes dermatitis herpetiformis?',
 'Who is at highest risk for Dermatitis herpetiformis ?',
 'When to seek urgent medical care when I have Dermatitis

Data Preprocessing

In [ ]:
for i in range(len(questions)):
  if questions[i] == "":
    print("hai")
    questions[i] = "none"

hai
hai


Exporting questions and answers list to csv file

In [ ]:
dataDict = {'Questions': questions, 'Answers': answers}
dataFrame = pd.DataFrame(dataDict)
dataFrame.to_csv('/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/queAns.csv')

Making the documents into chunks

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 5000)
docs = text_splitter.create_documents(questions)
len(docs)

6388

In [ ]:
docs

[Document(page_content='What are the symptoms of Allergy?'),
 Document(page_content='What causes Allergy?'),
 Document(page_content='When to seek urgent medical care when I have Allergy ?'),
 Document(page_content='What to expect if I have Allergy  (Outlook/Prognosis)?'),
 Document(page_content='What are the symptoms of Anaphylaxis?'),
 Document(page_content='What causes Anaphylaxis?'),
 Document(page_content='When to seek urgent medical care when I have Anaphylaxis ?'),
 Document(page_content='What to expect if I have Anaphylaxis  (Outlook/Prognosis)?'),
 Document(page_content='What are the symptoms of Angioedema?'),
 Document(page_content='What causes Angioedema?'),
 Document(page_content='When to seek urgent medical care when I have Angioedema ?'),
 Document(page_content='What to expect if I have Angioedema  (Outlook/Prognosis)?'),
 Document(page_content='What are the symptoms of Asthma?'),
 Document(page_content='What causes Asthma?'),
 Document(page_content='Who is at highest risk

Creating the embedding for similarity

In [ ]:
model_path = 'sentence-transformers/all-mpnet-base-v2'
encoder_args = {'device': 'cpu'}
encoder_test = {'normalize_embeddings': False}

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name = model_path,
                              model_kwargs = encoder_args,
                              encode_kwargs = encoder_test)

Exporting the embeddings

In [ ]:
import joblib
joblib.dump(embeddings, "/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/embeddings.pkl")

['/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/embeddings.pkl']

Testing the embeddings

In [ ]:
test_query = "What are the difficulties while treating fever"
test_embed = embeddings.embed_query(test_query)
len(test_embed)

768

Creating vector database for similarity search Using FAISS

In [ ]:
db = FAISS.from_documents(docs, embeddings)

In [ ]:
db.save_local("/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/faiss_db")

Testing the db

In [ ]:
query = "I'm suffereing from fever"
search_doc = db.similarity_search_with_relevance_scores(query, 4)
print(search_doc)

[(Document(page_content='When to seek urgent medical care when I have Q fever ?'), 0.4501942347348419), (Document(page_content='When to seek urgent medical care when I have Relapsing fever ?'), 0.4490574499368061), (Document(page_content='When to seek urgent medical care when I have Yellow fever ?'), 0.43797582120468515), (Document(page_content='When to seek urgent medical care when I have Viral hemorrhagic fever ?'), 0.4066097993680897)]


In [ ]:
%%writefile app.py

# Copyright (c) Streamlit Inc. (2018-2022) Snowflake Inc. (2022)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

import streamlit as st
from streamlit.logger import get_logger
from langchain.vectorstores import FAISS

import joblib

vectorDB = joblib.load("/content/drive/MyDrive/Colab Notebooks/GEN_AI-Intern/Practice-miniProjects/RAG-doctor-chatBot/db.faiss", mmap_mode='r')


LOGGER = get_logger(__name__)



st.title(" Doctor bot ")

if "messages" not in st.session_state:
  st.session_state.messages = []

"""for message in st.session_state.messages:
  with st.chat_message(message['role']):
    st.write(message["content"])"""

prompt = st.chat_input("What's up ?")

if prompt: # if input is given
  # updating user input to the chat container
  with st.chat_message("user"):
    st.markdown(f"Me: {prompt}")

  # appending to the chat history
  st.session_state.messages.append({"role": "user", "content": prompt})

  # getting the response and updating the chat container
  search_doc = vectorDB.similarity_search_with_relevance_scores(prompt, 4)
  with st.chat_message("assistant"):
    st.markdown(f"DocBot: {search_doc}")

  # Appending the chat history list
  st.session_state.messages.append({"role": "assistant", "content": prompt})

Writing app.py


In [ ]:
!streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.196.100.252:8501

  Stopping...
  Stopping...


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.441s
your url is: https://stale-bats-sniff.loca.lt


[Document(page_content='When to seek urgent medical care when I have Q fever ?'), Document(page_content='When to seek urgent medical care when I have Relapsing fever ?'), Document(page_content='When to seek urgent medical care when I have Yellow fever ?'), Document(page_content='When to seek urgent medical care when I have Viral hemorrhagic fever ?')]